In [1]:
import numpy as np
from multiprocessing import Pool
import networkx as nx
import random

In [65]:

def generate_signed_sbm_known_groups(groups, group_connectivity, p_plus):
    """
    A network from SBM model is generated. 
    `groups` contains sets of nodes. Each set of nodes is one group. 
    `groups` may also contain array of values, indicating number of nodes in each group with giving node indices. 
    `group_connectivity` is the connectivity between the groups. 
    `p_plus` are the probabilities of creating positive signs between and across groups. 
    
    Option 1: All groups are uniform, that is they have the same connectivity and positive link probabilities. 
    Then, `group_connectivity` and `p_plus` contain two values: `(din, dout)` and `(pin_plus, p_out_plus)`. 
    
    Option 2: All groups may be different. 
    Then, the dimension of `group_connectivity` and/or `p_plus` should be equal to the number of groups. 

    Args:
        groups (list): Array of arrays of nodes (when nodes' labels are important). Or array of group sizes. 
        group_connectivity (numpy array): Two values indicating in- and out-group connectivity. 
            Or array of size number of groups with specific connectivities. 
        p_plus (numpy array): Two values indicating pin+ and pout+ probabilities. 
            Or array of size number of groups with specific probabilities.
    """
    
    if np.isscalar(groups[0]):
        sizes = np.array(groups)
        actual_groups = [list(range(end_g - len_g,end_g)) for len_g, end_g in zip(sizes, np.cumsum(sizes))]
    else:
        actual_groups = groups
        sizes = np.array([len(g) for g in groups])
    N = np.sum(sizes)
    
    ng = len(sizes)
    # get dictionary which node belongs to which group. This will be useful in the last stage
    d = dict.fromkeys(range(0,N))
    for i, group in enumerate(actual_groups):
        for node in group:
            d[node] = i
    
    if len(group_connectivity) == 2:
        conn = np.ones((ng, ng)) * group_connectivity[1]
        np.fill_diagonal(conn, group_connectivity[0])
    else:
        conn = group_connectivity
    
    g = nx.stochastic_block_model(sizes, conn)
    A = nx.to_numpy_array(g)
    
    if len(p_plus) == 2:
        pos_probs = np.ones((ng, ng)) * p_plus[1]
        np.fill_diagonal(pos_probs, p_plus[0])
    else:
        pos_probs = p_plus
    
    for agent_i, agent_j in g.edges:
        group_i = d[agent_i]
        group_j = d[agent_j]
        
        sign = +1 if np.random.rand() < pos_probs[group_i, group_j] else -1
        A[agent_i, agent_j] = sign
        A[agent_j, agent_i] = sign
    
    return A

In [6]:
groups = [3, 4, 5]
group_connectivity = [0.8, 0.5]

p_plus = [0.8, 0.3]

In [7]:
generate_signed_sbm_known_groups(groups, group_connectivity, p_plus)

array([[ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -1., -1.],
       [ 1.,  0.,  1., -1.,  0., -1., -1.,  0.,  0., -1.,  1.,  1.],
       [ 0.,  1.,  0., -1.,  0., -1.,  0., -1.,  0., -1.,  0., -1.],
       [ 0., -1., -1.,  0.,  1.,  1., -1., -1.,  1.,  0., -1.,  0.],
       [-1.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0., -1., -1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 0., -1.,  0., -1.,  1.,  1.,  0.,  1.,  0., -1.,  0., -1.],
       [ 0.,  0., -1., -1.,  0.,  0.,  1.,  0., -1., -1.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  1.,  1.,  1.],
       [ 0., -1., -1.,  0.,  0.,  0., -1., -1.,  1.,  0.,  1., -1.],
       [-1.,  1.,  0., -1., -1.,  1.,  0.,  1.,  1.,  1.,  0., -1.],
       [-1.,  1., -1.,  0.,  0.,  0., -1.,  0.,  1., -1., -1.,  0.]])

In [18]:
groups = [[1, 3, 9], [2, 4, 7, 0, ], [5, 6, 10, 11]]
group_connectivity = [0.8, 0.5]

p_plus = [0.8, 0.3]

# This will throw error because the above group is missing node 8
generate_signed_sbm_known_groups(groups, group_connectivity, p_plus)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [19]:
groups = [[1, 3, 9], [2, 4, 7, 0, 8], [5, 6, 10, 11]]
group_connectivity = [0.8, 0.5]

p_plus = [0.8, 0.3]

# This will throw error because the above group is missing node 8
generate_signed_sbm_known_groups(groups, group_connectivity, p_plus)

array([[ 0., -1.,  1., -1.,  0., -1., -1., -1.,  1., -1.,  0., -1.],
       [-1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -1., -1.],
       [ 1.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  1.,  0., -1., -1.],
       [-1.,  1., -1.,  0., -1., -1., -1., -1.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0., -1.,  1.,  1.,  0.,  0., -1.],
       [-1.,  0.,  1., -1.,  0.,  0., -1., -1.,  0., -1.,  1., -1.],
       [-1.,  0.,  0., -1., -1., -1.,  0.,  1., -1.,  0., -1.,  1.],
       [-1.,  0.,  0., -1.,  1., -1.,  1.,  0.,  0., -1., -1.,  0.],
       [ 1.,  0.,  1.,  0.,  1.,  0., -1.,  0.,  0.,  1., -1.,  0.],
       [-1.,  0.,  0.,  1.,  0., -1.,  0., -1.,  1.,  0.,  0., -1.],
       [ 0., -1., -1.,  0.,  0.,  1., -1., -1., -1.,  0.,  0.,  1.],
       [-1., -1., -1.,  0., -1., -1.,  1.,  0.,  0., -1.,  1.,  0.]])

In [66]:
groups = [[1, 3, 9], [2, 4, 7, 0, 8], [5, 6, 10, 11]]
group_connectivity = [[0.8, 0.5, 0.3],[0.5,0.7,0.1], [0.3, 0.1, 0.5]]

p_plus = [0.8, 0.3]

# This will throw error because group connectivity should be symmetric
generate_signed_sbm_known_groups(groups, group_connectivity, p_plus)

array([[ 0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  1.,  0.,  0., -1.,  0.,  0., -1.],
       [ 1., -1.,  0.,  1.,  0.,  0., -1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  1.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  1., -1.,  0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0., -1.,  0.,  1., -1.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [67]:
groups = [[1, 3, 9], [2, 4, 7, 0, 8], [5, 6, 10, 11]]
group_connectivity = [[0.8, 0.5, 0.3],[0.5,0.7,0.1], [0.3, 0.1, 0.5]]

p_plus = np.array([[0.8, 0.3, 0.3],
        [0.3, 0.8, 0.3],
        [0.3, 0.3, 0.8]])

# This will throw error because group connectivity should be symmetric
generate_signed_sbm_known_groups(groups, group_connectivity, p_plus)

array([[ 0., -1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [-1.,  0., -1.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  1.,  0., -1.,  1.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0., -1.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0., -1.,  1.,  0., -1.,  1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  1., -1., -1.,  0., -1.,  0.],
       [ 0., -1., -1.,  1., -1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1., -1.,  1., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0., -1., -1.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -1., -1.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])